<a href="https://colab.research.google.com/github/emily5-1-2/Caltech101_Classifier/blob/main/Caltech101_Classifier_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchmetrics -qU
!pip install lightning -qU
!pip install ftfy regex tqdm -qU
!pip install git+https://github.com/openai/clip -qU
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 17.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchmetrics.functional import accuracy
import lightning as L
from pytorch_lightning.loggers import WandbLogger
from sklearn.model_selection import train_test_split
import clip
import wandb

In [ ]:
class Caltech101_Classifier(L.LightningModule):

    def __init__(self, CLIP_model, embed_dim=512, mlp_dim=200, num_classes=101, lr=1e-3):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_classes = num_classes
        self.CLIP_model = CLIP_model
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, mlp_dim),
            nn.ReLU(),
            nn.Linear(mlp_dim, num_classes)
        )
        self.lr = lr

    def forward(self,x):
        #Embed the images through CLIP
        x = self.CLIP_model.encode_image(x).float()
        x = x.reshape(-1, self.embed_dim)
        #Fully connected layers for classification
        x = self.mlp(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = self(x)
        #Compute cross entropy loss
        loss = nn.functional.cross_entropy(x, y)
        #Compute accuracy
        preds = torch.argmax(x, dim=1)
        acc = accuracy(preds, y, task="multiclass", num_classes=self.num_classes)
        #Log training loss and accuracy
        self.log("train_loss", loss)
        self.log("train_acc", acc)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = self(x)
        loss = nn.functional.cross_entropy(x, y)
        preds = torch.argmax(x, dim=1)
        acc = accuracy(preds, y, task="multiclass", num_classes=self.num_classes)
        #Log validation loss and accuracy
        self.log("val_loss", loss)
        self.log("val_acc", acc)

    def test_step(self, batch, batch_idx):
        x, y = batch
        x = self(x)
        preds = torch.argmax(x, dim=1)
        acc = accuracy(preds, y, task="multiclass", num_classes=self.num_classes)
        #Log testing accuracy
        self.log("test_acc", acc)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

In [ ]:
def train_and_test(mlp_dim=200, lr=1e-3, batch_size=64, group="Change layer size"):
    """Function for training a Caltech101_Classifier and logging the results to wandb.

    Keyword arguments:
    mlp_dim -- size of fully connected layer in the classifier (default 200)
    lr -- learning rate (default 1e-3)
    batch_size -- batch_size for training the model (default 64)
    group -- wandb group for the log (default "Change layer size")
    """

    #Start a new background process to log data to a run
    wandb.init(project='Caltech101',
        group=group,
        config={
            "model_name": "Caltech101_Classifier",
            "batch_size": batch_size,
            "mlp_dim": mlp_dim,
            "learning_rate": lr,
        })

    #Create dataloaders
    train_dataloader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dset, batch_size=batch_size, shuffle=True)

    #Set up wandb logger
    name = "Experiment: layer_size={mlp_dim}, lr={lr}, batch={batch_size}"
    wandb_logger = WandbLogger(project="Caltech101", name=name.format(mlp_dim=mlp_dim, lr=lr, batch_size=batch_size), log_model="all")

    #Create model based on the provided hyperparameters
    model = Caltech101_Classifier(CLIP_model=CLIP_model, mlp_dim=mlp_dim, lr=lr)
    model = model.to(device)

    #Train model for 10 epochs
    trainer = L.Trainer(max_epochs=10, logger=wandb_logger)
    trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

    wandb.finish()

In [ ]:
wandb.login()

True

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
CLIP_model, CLIP_preprocess = clip.load("ViT-B/32", device=device)

#Fix the CLIP model so that its weights are not updated during training
for param in CLIP_model.parameters():
    param.requires_grad = False

#Download Caltech101 and transform using the preprocess function of CLIP
dataset = datasets.Caltech101(root="/content/drive/MyDrive", transform=CLIP_preprocess, download=True)

#Split dataset into training and validation sets
train_dset, val_dset = train_test_split(dataset, train_size=0.85)

Files already downloaded and verified


In [ ]:
#Below is a simple experiment for varying the layer size
mlp_dims = [150, 200, 300, 400]
for mlp_dim in mlp_dims:
  train_and_test(mlp_dim=mlp_dim, group="Change layer size")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name       | Type       | Params
------------------------------------------
0 | CLIP_model | CLIP       | 151 M 
1 | mlp        | Sequential | 92.2 K
------------------------------------------
92.2 K    Trainable params
151 M     Non-trainable params
151 M     Tota

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


epoch,▁▁▁▂▂▂▃▃▃▃▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇████
train_acc,▁▅█▅▇▇█▆█▇█████████████
train_loss,█▄▁▂▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇███
val_acc,▁▄▆▆▇██▇▇█
val_loss,█▄▂▂▂▁▁▁▁▁
epoch,9
train_acc,1.0
train_loss,0.01369
trainer/global_step,1159
val_acc,0.9616


In [ ]:
#Below is a simple experiment for varying the learning rate
lrs = [1e-2, 1e-3, 1e-4, 1e-5]
for lr in lrs:
    train_and_test(lr=lr, group="Change learning rate")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name       | Type       | Params
------------------------------------------
0 | CLIP_model | CLIP       | 151 M 
1 | mlp        | Sequential | 122 K 
------------------------------------------
122 K     Trainable params
151 M     Non-trainable params
151 M     Total params
605.601   Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name       | Type       | Params
------------------------------------------
0 | CLIP_model | CLIP       | 151 M 
1 | mlp        | Sequential | 122 K 
---------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


epoch,▁▁▁▂▂▂▃▃▃▃▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇████
train_acc,▁▁▄▄▄▇▅▅▅▇▅▆▇▇▅▆▇▆▆███▆
train_loss,████▇▇▇▇▇▆▇▆▅▅▅▄▄▄▄▃▂▁▃
trainer/global_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇███
val_acc,▁▆▇▇▇█████
val_loss,█▇▇▆▅▄▄▃▂▁
epoch,9
train_acc,0.26562
train_loss,3.58769
trainer/global_step,1159
val_acc,0.28264


In [ ]:
#Below is a simple experiment for varying the batch size
batch_sizes = [16, 32, 64, 128]
for batch_size in batch_sizes:
    train_and_test(batch_size=batch_size, group="Change batch_size")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name       | Type       | Params
------------------------------------------
0 | CLIP_model | CLIP       | 151 M 
1 | mlp        | Sequential | 122 K 
------------------------------------------
122 K     Trainable params
151 M     Non-trainable params
151 M     Total params
605.601   Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name       | Type       | Params
------------------------------------------
0 | CLIP_model | CLIP       | 151 M 
1 | mlp        | Sequential | 122 K 
---------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▆▇▇██
train_acc,▁▅▇▇▇▇▇▇▇██
train_loss,█▃▂▂▂▁▁▁▁▁▁
trainer/global_step,▁▁▂▂▂▃▃▃▄▄▄▅▅▆▆▆▆▇▇██
val_acc,▁▆▇▇██████
val_loss,█▃▂▂▁▁▁▁▁▁
epoch,9
train_acc,1.0
train_loss,0.03096
trainer/global_step,579
val_acc,0.95929
